In [16]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

#### Fetch the data

In [17]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

website_url = requests.get(url).text
soup = BeautifulSoup(website_url, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})
rows = table.text.split('\n\n\n')

data = list()
for r in rows:
    items = r.replace('\n\n', '').split('\n')
    data.append(items)

df = pd.DataFrame(data=data[1:], columns=data[0])

In [18]:
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Clean the DataFrame

In [19]:
# Rename Postcode by PostalCode
df['PostalCode'] = df['Postcode']
del df['Postcode']

# Remove Borough = Not assigned
df = df[df['Borough'] != 'Not assigned']

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
mask = (df['Neighborhood'] == 'Not assigned')
df['Neighborhood'][mask] = df['Borough']

# Combined into one row with the neighborhoods separated with a comma
transformed_df = df.groupby(['PostalCode'])['Neighborhood'].apply(','.join).reset_index()
df = pd.merge(left=transformed_df, right=df[['PostalCode', 'Borough']], on='PostalCode', how='left')
df = df[['PostalCode', 'Borough', 'Neighborhood']]
df.drop_duplicates(inplace=True)

In [20]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
2,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
5,M1E,Scarborough,"Guildwood,Morningside,West Hill"
8,M1G,Scarborough,Woburn
9,M1H,Scarborough,Cedarbrae


#### Print the shape

In [9]:
print(df.shape)

(103, 3)
